In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
import torchio as tio

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from segmentation_pipeline import *


if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [2]:
prediction_name = "nnunet_postprocessed"

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern="ground_truth.*", image_name="ground_truth", 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
transform = CustomOneHot()
dataset = SubjectFolder("X:/Datasets/MSSEG2_processed/", subject_path="", subject_loader=subject_loader)

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern=f"{prediction_name}.*", image_name=prediction_name, 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
dataset.load_additional_data('X:/Predictions/MSSEG2/', subject_loader)

In [3]:
seg_eval = InstanceSegmentationEvaluator(prediction_name, 'ground_truth')
evaluation = seg_eval(dataset)
evaluation['summary_stats'] = evaluation['summary_stats'].to_dataframe()

C:\Users\Cefir\anaconda3\envs\machine_learning\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [4]:
evaluation['subject_stats']

,subject,target_components,predicted_components,target_detections,predicted_detections,detection_recall,detection_precision,detection_f1,target_volume,prediction_volume,TP,FP,TN,FN,dice,jaccard,precision,recall
0,013,1.0,1.0,0.0,0.0,0.000000,0.000000,NaN,40.0,39.0,0.0,39.0,23920560.0,40.0,0.000000,0.000000,0.000000,0.000000
1,015,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,23920640.0,0.0,NaN,NaN,NaN,NaN
2,016,7.0,4.0,3.0,3.0,0.428571,0.750000,0.545455,743.0,536.0,482.0,54.0,23919844.0,261.0,0.753714,0.604768,0.899254,0.648721
3,018,2.0,0.0,0.0,0.0,0.000000,NaN,NaN,364.0,0.0,0.0,0.0,18431636.0,364.0,0.000000,0.000000,NaN,0.000000
4,019,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,83886080.0,0.0,NaN,NaN,NaN,NaN
5,020,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,201.0,154.0,131.0,23.0,40845088.0,70.0,0.738028,0.584821,0.850649,0.651741
6,021,4.0,6.0,4.0,4.0,1.000000,0.666667,0.800000,2227.0,3141.0,2041.0,1100.0,23917312.0,186.0,0.760432,0.613466,0.649793,0.916480
7,024,6.0,5.0,5.0,5.0,0.833333,1.000000,0.909091,703.0,760.0,582.0,178.0,23919760.0,121.0,0.795625,0.660613,0.765789,0.827881
8,026,9.0,8.0,7.0,4.0,0.777778,0.500000,0.608696,376.0,241.0,173.0,68.0,5307972.0,203.0,0.560778,0.389640,0.717842,0.460106
9,027,4.0,5.0,4.0,4.0,1.000000,0.800000,0.888889,1059.0,785.0,740.0,45.0,23919536.0,319.0,0.802603,0.670290,0.942675,0.698772


In [5]:
evaluation['summary_stats']

,summary_stat,target_components,predicted_components,target_detections,predicted_detections,detection_recall,detection_precision,detection_f1,target_volume,prediction_volume,TP,FP,TN,FN,dice,jaccard,precision,recall
0,mean,3.800000,3.200000,2.375000,2.225000,0.553623,0.700968,0.766852,1485.800049,1236.025024,997.174988,238.850006,33059556.0,488.625000,0.500586,0.403031,0.710442,0.481444
1,std,6.013659,4.804912,3.746366,3.555332,0.403918,0.348857,0.231320,3778.051270,3104.793457,2701.393311,590.721802,19049108.0,1243.758057,0.357561,0.302365,0.313231,0.343216
2,min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.235294,0.000000,0.000000,0.000000,0.000000,5307972.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,max,32.000000,22.000000,18.000000,18.000000,1.000000,1.000000,1.000000,19704.000000,15252.000000,13961.000000,2522.000000,83886080.0,5743.000000,0.909984,0.834835,1.000000,0.978873
4,median,1.000000,1.000000,1.000000,1.000000,0.666667,0.800000,0.800000,201.000000,39.000000,19.000000,6.000000,23920560.0,104.000000,0.680518,0.515747,0.849039,0.581152
5,mode,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,23920640.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
import wandb
from segmentation_pipeline.loggers.wandb_logger import to_wandb

wandb.init(project="msseg2-evaluation", dir="X:/wandb/", name=prediction_name)
wandb.log(to_wandb(evaluation))
wandb.finish()

wandb: Currently logged in as: efirdc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


_runtime,6
_timestamp,1630813583
_step,0


_runtime,▁
_timestamp,▁
_step,▁
